In [ ]:
from cProfile import label
from re import X
from turtle import color
from sympy import li
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as D
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
import importlib

import dual_enc_dec_cnmp as dual_enc_dec_cnmp
import utils
importlib.reload(dual_enc_dec_cnmp)
importlib.reload(utils)

#params = np.array([[2,0],[2,0],[2,0],[2,0],[2,0],[-0.6,0.1],[-0.55, 0.16],[-0.5,0.23],[-0.45, 0.33],[-0.4,0.43]])
params = np.array([[20],[19.9],[19.7]])

num_demo = 128
X1, X2, Y1, Y2, validation_Y1, validation_Y2 = utils.generate_demonstrations(num_demo, time_len=200, params = params, plot_title='Training')

In [3]:
demo_data = [X1, X2, Y1, Y2]
d_x = 1
d_y1 = 1
d_y2 = 1
OBS_MAX = 30
d_N = num_demo
time_len = 200

validation_indices_1 = [i+1 for i in range(0, 32, 7)]
validation_indices_2 = [i+2 for i in range(0, 32, 7)]
validation_indices = validation_indices_1 + validation_indices_2

In [4]:
importlib.reload(dual_enc_dec_cnmp)
importlib.reload(utils)


errors = []
losses = []

def train():

    model = dual_enc_dec_cnmp.DualEncoderDecoder(d_x, d_y1, d_y2).double()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    def lr_lambda(epoch):
        if epoch < 50000:
            return 1e-3 / optimizer.defaults['lr']  # Scale factor for 1e-3
        elif 50000 <= epoch < 100000:
            return 1e-4 / optimizer.defaults['lr']  # Scale factor for 1e-4
        else:
            return 1e-5 / optimizer.defaults['lr']  # Scale factor for 1e-5
    
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    error = 0

    for i in range(EPOCHS):

        obs, x_tar, y_tar = dual_enc_dec_cnmp.get_training_sample(validation_indices, X1, Y1, X2, Y2, OBS_MAX, d_N, d_x, d_y1, d_y2, time_len)
        #print(np.array(x_tar).shape)
        optimizer.zero_grad()
        output = model(obs, x_tar)
        #print(np.array(y_tar).shape)
        loss = dual_enc_dec_cnmp.log_prob_loss(output, y_tar, d_y1)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        if i % 100 == 0:
            if i % 100 == 0:
                
                # print(f'Epoch: {i}, Loss: {loss.item()}')
                error = 0
                plot_id = np.random.randint(0, len(validation_indices))
                for validation_idx in validation_indices:
                    time = np.linspace(0, 1, time_len)
                    # permute time
                    idx = np.random.permutation(time_len)
                    idx = idx[:OBS_MAX]
                    time = [time[i] for i in idx]
                    condition_points = [[t, Y1[validation_idx, i:i+1]] for t,i in zip(time, idx)]
                    means, stds = dual_enc_dec_cnmp.predict_inverse(model, validation_idx, time_len, condition_points, d_x, d_y1, d_y2, demo_data)
                    if i % 10000 == 0 and validation_idx == validation_indices[plot_id]:
                        error += utils.validate_model(means, stds, validation_idx, demo_data, time_len, condition_points, i, plot=True)
                    else:
                        error += utils.validate_model(means, stds, validation_idx, demo_data, time_len, condition_points, i, plot=False)
                errors.append(error)
                if error == min(errors):
                    print('Saving model ', i)
                    torch.save(model.state_dict(), 'inverse_best_model.pth')
                
                
                losses.append(loss.item())
                """
                if loss.item() == min(losses):
                    print('Saving model ', i)
                    torch.save(model.state_dict(), 'inverse_best_model.pth')
                """
                
                continue

            losses.append(loss.item())
            
            """
            if loss.item() == min(losses):
                print('Saving model ', i)
                torch.save(model.state_dict(), 'inverse_best_model.pth')
            """
    

In [ ]:
importlib.reload(dual_enc_dec_cnmp)
importlib.reload(utils)

EPOCHS = 500000
learning_rate = 1e-3

train()

In [ ]:
importlib.reload(dual_enc_dec_cnmp)
importlib.reload(utils)

best_model = dual_enc_dec_cnmp.DualEncoderDecoder(d_x, d_y1, d_y2).double()
best_model.load_state_dict(torch.load('inverse_best_model.pth'))

t = 0.25
min_y_f = np.min(Y1[:, int(t * time_len)])
max_y_f = np.max(Y1[:, int(t * time_len)])
interval_y_f = np.linspace(min_y_f-0.1, max_y_f+0.1, 20)

min_y_i = np.min(Y2[:, int(t * time_len)])
max_y_i = np.max(Y2[:, int(t * time_len)])
interval_y_i = np.linspace(min_y_i-0.1, max_y_i+0.1, 20)

i = 0
for idx in interval_y_f:
    condition_points = [[t, idx]]
    means, stds = dual_enc_dec_cnmp.predict_inverse(best_model, 1, time_len, condition_points, d_x, d_y1, d_y2, demo_data)
    if i == 0:
        utils.plot_results(means, stds, Y1, Y2, idx, condition_points, errors, losses, time_len, d_N, plot_errors = True)
        i += 1
    utils.plot_results(means, stds, Y1, Y2, idx, condition_points, errors, losses, time_len, d_N, plot_errors = False)


In [ ]:
importlib.reload(dual_enc_dec_cnmp)
importlib.reload(utils)

best_model = dual_enc_dec_cnmp.DualEncoderDecoder(d_x, d_y1, d_y2).double()
best_model.load_state_dict(torch.load('inverse_best_model.pth'))


num_validate = 10
num_obs = 30
freq = 0.
def dist(amplitude, x):
    #return amplitude * math.sin(2 * torch.pi * freq * x + 0)
    if x<0.5: #or True:
        return -amplitude * x + amplitude * (1-x)
    else:
        return -amplitude * math.sin(2 * math.pi * 2 * x + 0)
    
amplitudes = np.linspace(0.75,1.0,num_validate)
times = np.linspace(0,1,num_obs)

condition_point_set = []
for i in range(num_validate):
    condition_point_set.append([])
    for j in range(num_obs):
        condition_point_set[-1].append([times[j], dist(amplitudes[i], times[j])])




for i in range(len(condition_point_set)):
    condition_points = condition_point_set[i]
    means, stds = dual_enc_dec_cnmp.predict_inverse(best_model, 0, time_len, condition_points, d_x, d_y1, d_y2, demo_data)
    if i == 0:
        utils.plot_results(means, stds, Y1, Y2, idx, condition_points, errors, losses, time_len, d_N, plot_errors = True, test_dist =(dist,i,num_validate))
    utils.plot_results(means, stds, Y1, Y2, idx, condition_points, errors, losses, time_len, d_N, plot_errors = False, test_dist = (dist,i,num_validate))


In [ ]:

importlib.reload(dual_enc_dec_cnmp)
importlib.reload(utils)
from mpl_toolkits.mplot3d import Axes3D


t = 0.25
min_y_f = np.min(Y1[:, int(t * time_len)])
max_y_f = np.max(Y1[:, int(t * time_len)])
interval_y_f = np.linspace(min_y_f, max_y_f, 20)

min_y_i = np.min(Y2[:, int(t * time_len)])
max_y_i = np.max(Y2[:, int(t * time_len)])
interval_y_i = np.linspace(min_y_i, max_y_i, 20)

## perform PCA on the latent space
observations_f = torch.tensor([[t, y] for y in interval_y_f])
observations_i = torch.tensor([[t, y] for y in interval_y_i])

pca_result = dual_enc_dec_cnmp.plot_latent_space(best_model, observations_f, observations_i)

## open 2 subplots
fig, axs = plt.subplots(1, 3, figsize=(20, 7))

## add a horizontal line at t = 0.25
for i in range(2):
    axs[i].axvline(x=t, color='r', linestyle='--')

for forward_traj in Y1:
    axs[0].plot(np.linspace(0, 1, time_len), forward_traj, c='black', alpha=0.2)
for inverse_traj in Y2:
    axs[1].plot(np.linspace(0, 1, time_len), inverse_traj, c='black', alpha=0.2)
axs[0].set_title("Forward trajectories and observations")
axs[1].set_title("Inverse trajectories and observations")
for i in range(len(interval_y_f) + len(interval_y_i)):
    point = pca_result[i]
    if i < 20:
        if i == 0:
            axs[0].scatter(t, interval_y_f[i], c=f"{(i)/(1.4*len(interval_y_f))}", label = f"Forward observation, smallest amplitude at t = {t}")
            continue
        if i == 19:
            axs[0].scatter(t, interval_y_f[i], c=f"{(i)/(1.4*len(interval_y_f))}", label = f"Forward observation, greatest amplitude at t = {t}")
            continue
        axs[0].scatter(t, interval_y_f[i], c=f"{(i)/(1.4*len(interval_y_f))}")#, label = f"forward")
    else:
        if i == 20:
            axs[1].scatter(t, interval_y_i[i-20], c=f"{(39-i)/(1.4*len(interval_y_f))}", marker="^", label = f"Inverse observation, smallest amplitude at t = {t}")
            continue
        if i == 39:
            axs[1].scatter(t, interval_y_i[i-20], c=f"{(39-i)/(1.4*len(interval_y_f))}", marker="^", label = f"Inverse observation, greatest amplitude at t = {t}")
        axs[1].scatter(t, interval_y_i[i-20], c=f"{(39-i)/(1.4*len(interval_y_f))}", marker="^")#, label = f"inverse")

# plot 3d
for i in range(len(interval_y_f) + len(interval_y_i)):
    point = pca_result[i]
    #plot 3d
    if i < 20:
        if i == 0:
            axs[2].scatter(point[0], point[1], c=f"{(i)/(1.4*len(interval_y_f))}", label = f"Forward observation, smallest amplitude at t = {t}")
            continue
        if i == 19:
            axs[2].scatter(point[0], point[1], c=f"{(i)/(1.4*len(interval_y_f))}", label = f"Forward observation, greatest amplitude at t = {t}")
            continue
        plt.scatter(point[0], point[1], c=f"{(i)/(1.4*len(interval_y_f))}")#, label = f"forward")
    else:
        if i == 20:
            axs[2].scatter(point[0], point[1], c=f"{(39-i)/(1.4*len(interval_y_f))}", marker="^", label = f"Inverse observation, greatest amplitude at t = {t}")
            continue
        if i == 39:
            axs[2].scatter(point[0], point[1], c=f"{(39-i)/(1.4*len(interval_y_f))}", marker="^", label = f"Inverse observation, smallest amplitude at t = {t}")
        axs[2].scatter(point[0], point[1], c=f"{(39-i)/(1.4*len(interval_y_f))}", marker="^")#, label = f"inverse")

plt.title("PCA Analysis of the latent space of a single observation at t = 0.25")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

